#TP3

Base de dados de uma empresa de Telecom que oferta diversos serviços, entre os principais: internet e telefone.

Ao analisar o histórico dos clientes dos últimos anos percebe-se que a empresa está com Churn (taxa de cancelamento) de mais de 26% dos clientes. O trabalho consiste em encontrar uma (ou mais regras) que seja o mais acurada possível para determinar os clientes com churn (os clientes que cancelaram). Deve-se utilizar algoritmos genéticos ou programação genética para encontrar tal regra.

Link no Kaggle: https://www.kaggle.com/radmirzosimov/telecom-users-dataset

In [100]:
# Passo 1: Importar a base de dados
import pandas as pd

tabela = pd.read_csv("telecom_users.csv")

In [101]:
# Passo 2: Visualizar a base de dados
# Entender quais informações estão disponíveis
# Retirada da primeira coluna
tabela = tabela.drop("Unnamed: 0", axis=1) # 0 é linha e 1 é coluna

In [102]:
# Passo 3: Tratamento da base de dados
# Valores que são números mas que o Python acha que são texto
tabela["TotalGasto"] = pd.to_numeric(tabela["TotalGasto"], errors="coerce")

# Valores que estão vazios
# colunas vazias
# any para excluir colunas que tem PELO MENOS 1 valor vazio
# all para excluir colunas COMPLETAMENTE vazias
tabela = tabela.dropna(how="all", axis=1) 

# linhas vazias
tabela = tabela.dropna(how="any", axis=0) 

# note que a limpeza excluiu 12 registros
print(tabela.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5974 entries, 0 to 5985
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   IDCliente               5974 non-null   object 
 1   Genero                  5974 non-null   object 
 2   Aposentado              5974 non-null   int64  
 3   Casado                  5974 non-null   object 
 4   Dependentes             5974 non-null   object 
 5   MesesComoCliente        5974 non-null   int64  
 6   ServicoTelefone         5974 non-null   object 
 7   MultiplasLinhas         5974 non-null   object 
 8   ServicoInternet         5974 non-null   object 
 9   ServicoSegurancaOnline  5974 non-null   object 
 10  ServicoBackupOnline     5974 non-null   object 
 11  ProtecaoEquipamento     5974 non-null   object 
 12  ServicoSuporteTecnico   5974 non-null   object 
 13  ServicoStreamingTV      5974 non-null   object 
 14  ServicoFilmes           5974 non-null   

In [103]:
# Passo 4: Análise Exploratória -> Análise Geral -> Ver como estão os cancelamentos

display(tabela["Churn"].value_counts())
display(tabela["Churn"].value_counts(normalize=True).map("{:.1%}".format))

Nao    4387
Sim    1587
Name: Churn, dtype: int64

Nao    73.4%
Sim    26.6%
Name: Churn, dtype: object

## Encontramos linhas com valor nulo na coluna de dependentes e churn. Célula abaixo irá remove-los

In [104]:
tabela = tabela.dropna(subset=["Dependentes"])
tabela = tabela.dropna(subset=["Churn"])

In [105]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder

## Na célula abaixo, iremos converter todas as colunas que possuem valores binários, do tipo ("sim", "nao") por exemplo, em valores inteiros 0 ou 1

In [110]:
tabela1 = pd.DataFrame()
enc = LabelEncoder()
to_enc = tabela["Genero"]
enc.fit(to_enc)
tabela["Genero"] = enc.transform(tabela["Genero"])
to_enc = tabela["Aposentado"]
enc.fit(to_enc)
tabela["Aposentado"] = enc.transform(tabela["Aposentado"])
to_enc = tabela["Casado"]
enc.fit(to_enc)
tabela["Casado"] = enc.transform(tabela["Casado"])
to_enc = tabela["Dependentes"]
enc.fit(to_enc)
tabela["Dependentes"] = enc.transform(tabela["Dependentes"])
to_enc = tabela["ServicoTelefone"]
enc.fit(to_enc)
tabela["ServicoTelefone"] = enc.transform(tabela["ServicoTelefone"])
to_enc = tabela["FaturaDigital"]
enc.fit(to_enc)
tabela["FaturaDigital"] = enc.transform(tabela["FaturaDigital"])

to_enc = tabela["Churn"]
enc.fit(to_enc)
tabela["Churn"] = enc.transform(tabela["Churn"])


## Aqui separamos em conjunto de treino e de teste

In [116]:
X_train, X_test, Y_train, Y_test = train_test_split(
    tabela.drop("Churn", axis=1),
    tabela["Churn"],
    test_size=0.2, random_state=0
)